# Check Trends On Real DATA 

In [16]:
import numpy as np
import sympy as sp
import os

In [3]:
# Open Trends Data
TrendLib = []
with open(r"TheoreticalExperiments\TwoShot\c.log", "r+", encoding="utf-8") as file:
    for trend in file.readlines(): 
        for ch in range(len(trend)):
            if trend[ch] == " ":
                trend_without_last_moves = trend[:ch -2]
                clean_trend = trend_without_last_moves.split(':')[-1]
                TrendLib.append(clean_trend) # append and remove last two moves
                break


print(TrendLib)

['↓↓', '↓*↓', '↓↓↓', '↓**↓', '↓*↓↓', '↓↓*↓', '↓↓↑↓', '↓↓↓↓', '↓***↓', '↓**↓↓', '↓*↓*↓', '↓*↓↑↓', '↓*↓↓*', '↓*↓↓↓', '↓↑↓*↓', '↓↑↓↓↓', '↓↓**↓', '↓↓*↑↓', '↓↓*↓*', '↓↓*↓↓', '↓↓↓**', '↓↓↓*↓', '↓↓↓↑↓', '↓↓↓↓*', '↓↓↓↓↓', '↓****↓', '↓***↓↓', '↓**↓*↓', '↓**↓↑↓', '↓**↓↓↓', '↓*↓**↓', '↓*↓*↑↓', '↓*↓*↓↓', '↓*↓↓**', '↓*↓↓*↓', '↓*↓↓↑↓', '↓*↓↓↓*', '↓*↓↓↓↓', '↓↑↓↓*↓', '↓↓***↓', '↓↓**↑↓', '↓↓**↓*', '↓↓**↓↓', '↓↓*↓**', '↑↓*↓*↓', '↓↓*↓*↓', '↓↓*↓↑↓', '↓↓*↓↓*', '↑↓*↓↓↓', '↓↓*↓↓↓', '↓↓↑↓*↓', '↓↓↑↓↓↓', '↓↓↓***', '↑↓↓**↓', '↓↓↓**↓', '↓↓↓*↑↓', '↓↓↓*↓*', '↑↓↓*↓↓', '↓↓↓*↓↓', '↓↓↓↑↑↓', '↓↓↓↓**', '↑↓↓↓*↓', '↓↓↓↓*↓', '↓↓↓↓↑↓', '↓↓↓↓↓*', '↑↓↓↓↓↓', '↓↓↓↓↓↓', '↓*****↓', '↓****↓↓', '↓***↓*↓', '↓***↓↑↓', '↓***↓↓*', '↓***↓↓↓', '↓**↓**↓', '↓**↓*↑↓', '↓**↓*↓↓', '↓**↓↓**', '↓**↓↓*↓', '↓**↓↓↑↓', '↓**↓↓↓*', '↓**↓↓↓↓', '↓*↓***↓', '↓*↓**↑↓', '↓*↓**↓*', '↓*↓**↓↓', '↓*↓*↓**', '↓*↓*↓*↓', '↓*↓*↓↑↓', '↓*↓*↓↓*', '↓*↓*↓↓↓', '↓*↓↑*↓↓', '↓*↓↑↓*↓', '↓*↓↑↓↓↓', '↓*↓↓***', '↓*↓↓**↓', '↓*↓↓*↑↓', '↓*↓↓*↓*', '↓*↓↓*↓↓', '↓*↓↓↑*↓', '↓*↓↓↑↑↓', '↓*

In [24]:
# Preloads and Prerequirments - setting up
fx = np.loadtxt(r"Data\EURUSD\DAT_MT_EURUSD_M1_2020.csv", delimiter=",", dtype=str)
fx = fx[:, 5] # get 5th column of data
fx = fx.astype(np.float64)
TFfx = fx[:-1] < fx[1:]

print(TFfx)

def DECISIONMAKER (PastData):
    TFPastData = TFfx[:len(PastData) - 1]
    for trend in TrendLib:
        if len(trend) > len(TFPastData):
            continue
        for (num, ChrTF) in zip(range(len(trend)), reversed(trend)):
            if ChrTF == "*":
                continue
            if ChrTF != "↓↑"[TFPastData[-1-num]]: 
                break
        else:
            return False
            
    return None

[ True  True False ... False False False]


In [25]:
# One Time Bid - No Second Rebound
SELECTED = []
WIN = 0
LOSE = 0
TIE = 0
LEVEL = 0

for CurrentPriceIndex in range(len(fx)):
    if LEVEL == 0:
        PROPHET = DECISIONMAKER(fx[:CurrentPriceIndex + 1])
        if PROPHET is None:
            continue
        LEVEL += 1
    elif LEVEL == 1:
        SELECTED.append(fx[CurrentPriceIndex])
        LEVEL += 1
    elif LEVEL == 2:
        if not ((fx[CurrentPriceIndex] > SELECTED[0]) ^ PROPHET):
            WIN += 1
        elif SELECTED[0] == fx[CurrentPriceIndex]:
            TIE += 1
        else:
            LOSE += 1
        SELECTED = []
        LEVEL = 0

print(f"""win = {WIN} 
lose = {LOSE}
tie = {TIE}""")


C:\Users\MSI\AppData\Local\Temp\ipykernel_18876\1857119163.py:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  if ChrTF != "↓↑"[TFPastData[-1-num]]:


win = 65250 
lose = 56859
tie = 0


In [4]:
# Do nothing if First Bid changes direction
SELECTED = []
WIN = 0
LOSE = 0
TIE = 0
P_LOSE = 0
D_WIN = 0

for CurrentPriceIndex in range(len(fx)):
    if len(SELECTED) == 0:
        PROPHET = DECISIONMAKER(fx[:CurrentPriceIndex + 1])
        if PROPHET is None:
            continue
        SELECTED.append(fx[CurrentPriceIndex])
    elif len(SELECTED) == 1:
        if not ((fx[CurrentPriceIndex] > SELECTED[0]) ^ PROPHET):
            SELECTED.append(fx[CurrentPriceIndex])
        else:
            SELECTED.append(None)
    elif len(SELECTED) == 2:
        if SELECTED[1] == None:
            if not ((fx[CurrentPriceIndex] > SELECTED[0]) ^ PROPHET):
                WIN += 1
            elif SELECTED[0] == fx[CurrentPriceIndex]:
                TIE += 1
            else:
                LOSE += 1
        else:
            if SELECTED[1] == fx[CurrentPriceIndex]:
                TIE += 1
            elif SELECTED[0] == fx[CurrentPriceIndex]:
                TIE += 1
            elif not (( fx[CurrentPriceIndex] > SELECTED[1]) ^ 
                        ( fx[CurrentPriceIndex] > SELECTED[0])):
                P_LOSE += 1
            elif (( fx[CurrentPriceIndex] > SELECTED[1]) ^ 
                        ( fx[CurrentPriceIndex] > 
                        SELECTED[0])) and (( fx[CurrentPriceIndex]
                         > SELECTED[1]) ^ PROPHET):
                D_WIN += 1
        
        SELECTED = []

print(f"""win = {WIN} 
lose = {LOSE}
tie = {TIE}
p_lose = {P_LOSE}
d_win = {D_WIN}""")



C:\Users\MSI\AppData\Local\Temp\ipykernel_27000\3326298328.py:14: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  if ChrTF != "↓↑"[TFPastData[-1-num]]:


win = 24631 
lose = 69244
tie = 50986
p_lose = 89911
d_win = 13022
